# 1 Введение в Beautiful Soup

In [ ]:
import requests

# Ответ, содержит заголовки, код статуса, куки, тело ответа и т.п.
response = requests.get("https://wikipedia.org")

# Тело ответа в текстовом виде
html = response.text

In [ ]:
import re

# Поиск тэгов с использованием довольно сложной и ненадежной регулярки
re.findall(r'<a[^>]*other-project-link[^>]*href="([^"]*)', html)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

# Поиск тэгов с использованием Beautiful Soup
tags = soup('a', 'other-project-link')
for tag in tags:
    print(tag['href'])

In [ ]:
# Такой же поиск тэгов, но используем списковое включение
tags = [tag['href'] for tag in soup('a', 'other-project-link')]

for tag in tags:
    print(tag)

# 2 Обзор методов модуля Beautiful Soup

In [ ]:
from bs4 import BeautifulSoup

# Исходный HTML
html = """<!DOCTYPE html>
<html lang="en">
  <head>
    <title>test page</title>
  </head>
  <body class="mybody" id="js-body">
    <p class="text odd">first <b>bold</b> paragraph</p>
    <p class="text even">second <a href="https://mail.ru">link</a></p>
    <p class="list odd">third <a id="paragraph"><b>bold link</b></a></p>
  </body>
</html>
"""

# Суп, распарсенный с помощью LXML
soup = BeautifulSoup(html, 'lxml')

# Выведет текстовое представление супа
print(soup)

# Prettied-представление супа
# print(soup.prettify())

In [ ]:
print(soup.p)                            # Обратимся к первому тэгу p в супе
print(type(soup.p))                      # Тип данных тэга p - bs4.element.Tag
print(type(soup.p.b))                    # Тип данных тэга b - bs4.element.Tag
print(type(soup.p.b.string))             # Тип данных строки - bs4.element.NavigableString
print('Строка:',   soup.p.b.string)      # Казалось бы выглядит как обычная строка
print('Тэг:',      soup.b.name)          # Название тэга
print('Тэг:',      soup.p.name)          # Название тэга
print('Классы:',   soup.p['class'])      # Список классов (список или строка - зависит от спецификации атрибута)
print('Классы:',   soup.body['class'])   # Список классов (даже если класс единственный)
print('ID:',       soup.body['id'])      # ID body - всегда строка!
print('Родитель:', soup.p.b.parent)      # Родитель у <b> это <p>
print([t.name for t in soup.p.parents ]) # Родительские тэги
print('Следующий:',soup.p.next)          # Следующий (со спуском)
print('Сиблинг:',  soup.p.next_sibling)  # Выведет символ переноса строки, т.к. текст с переносами
print('Содержимое',soup.p.contents)      # Список вложенных элементов
print('Итератор:', soup.p.children)      # Список вложенных элементов (итератор)

# 3 Сложный поиск и изменение с Beautiful Soup


In [ ]:
print(soup.p.b.find_parent(id="js-body").name)         # Найти не просто родителя, а родителя с id = "js-body"
print(soup.p.b.find_parent("body")['id'])              # Найти среди родителей тэг body и вывести его id
print(soup.p.find_next_sibling(class_="odd"))          # Найти сиблинга нужного типа и с нужным классом
print(soup.p.find_next_siblings())                     # Список сиблингов 
print(soup.p.find('b'))                                # Найти тэг <b>
print(soup.p.find('b', text='bold'))                   # Найти тэг <b> только такой который с текстом bold
print(soup.find_all('p'))                              # Найти все тэги <p>
print(soup.find_all('p', 'text odd'))                  # Поиск по тэгу и классу
print(soup.find_all('p', 'odd text'))                  # Сменили порядок классов и ничего не находим....
print(soup.find_all(name='p', class_='text odd'))      # CSS-селектор, поиск по классам
print(soup.find_all(name='p', class_='odd text'))      # CSS-селектор, поиск по классам
print(soup.select('p.odd'))                            # CSS-селектор, выборка всех p.odd
print(soup.select('p:nth-of-type(3)'))                 # CSS-селектор, выведем третий тэг <p>
print(soup.select('a > b'))                            # CSS-селектор, ищем прямого потомка

import re
[i.name for i in soup.find_all(name=re.compile('^b'))] # Все тэги начинающиеся с b
[i for i in soup(['a', 'b'])]                          # Все тэги a и b - это уже без регулярок, просто список

In [ ]:
# Редактируем!
tag = soup.b
tag.name = 'i'
tag['id'] = 'myid'
tag.string = 'italic'

# Измененная верстка, можно сохранить например.
soup

# 4 Реальный пример - парсинг сайта

In [ ]:
# Соберем список секций и новостей в каждой из них

from bs4 import BeautifulSoup
import requests
result = requests.get("https://news.mail.ru/")
html = result.text
soup = BeautifulSoup(html, 'lxml')

In [ ]:
[
    (section.string, 
    [
        link.string for link in section.find_parents()[4].find_all('span', 'link__text')
    ] 
    ) for section in soup.find_all('span', 'hdr__inner')
]

# Домашнее задание - песочница

In [3]:
from bs4 import BeautifulSoup
import unittest

def parse(path_to_file):
    with open(path_to_file, encoding='utf-8') as f:
        html = f.read()
        
    soup = BeautifulSoup(html, 'lxml')
    root = soup.find('div', id='bodyContent')
        
    imgs = len(root.find_all(
        lambda tag:tag.name == "img" and
        "width" in tag.attrs and 
        int(tag["width"]) >= 200
    ))
    
    headers_first = body.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    headers_first = [header.text for header in headers_first]
    headers = []
    for i in range(len(headers_first)):
        if headers_first[i][0] in ['E','T','C']:
            headers.append(headers_first[i])
    headers = len(headers)
    
    lengths = []   
    for link in root.find_all('a'):
        counter = 1
        for l in link.find_next_siblings():
            if l.name  != 'a':
                break
            counter += 1
        lengths.append(counter)
    linkslen = max(lengths)
        
    lists = 0
    for listitem in root.find_all(['ul', 'ol']):
        if not listitem.find_parent('li'):
            lists += 1                                 
    
    return [imgs, headers, linkslen, lists]

class TestParse(unittest.TestCase):
    def test_parse(self):
        test_cases = (
            ('wiki/Stone_Age', [13, 10, 12, 40]),
            ('wiki/Brain', [19, 5, 25, 11]),
            ('wiki/Artificial_intelligence', [8, 19, 13, 198]),
            ('wiki/Python_(programming_language)', [2, 5, 17, 41]),
            ('wiki/Spectrogram', [1, 2, 4, 7]),)

        for path, expected in test_cases:
            with self.subTest(path=path, expected=expected):
                self.assertEqual(parse(path), expected)

if __name__ == '__main__':
    # unittest.main()
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # Run unit-tests in Jupyter

    
    

    

.
ERROR: test_build_bridge (__main__.TestGetStatistics) (path='wiki/', start_page='Stone_Age', end_page='Python_(programming_language)', expected=['Stone_Age', 'Brain', 'Artificial_intelligence', 'Python_(programming_language)'])
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-d4529b540632>", line 112, in test_build_bridge
    result = get_statistics(path, start_page, end_page)
  File "<ipython-input-1-d4529b540632>", line 88, in get_statistics
    stats = parse(fpath)
  File "<ipython-input-3-0765cb173301>", line 17, in parse
    headers_first = body.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
NameError: name 'body' is not defined

ERROR: test_build_bridge (__main__.TestGetStatistics) (path='wiki/', start_page='The_New_York_Times', end_page='Stone_Age', expected=['The_New_York_Times', 'London', 'Woolwich', 'Iron_Age', 'Stone_Age'])
---------------------------------------------------------------------

In [1]:
# Набор тестов для проверки студентами решений по заданию "Практическое задание
# по Beautiful Soup - 2". По умолчанию файл с решением называется solution.py,
# измените в импорте название модуля solution, если файл с решением имеет другое имя.

import os
import re
import unittest
from collections import deque
from solution_part1 import parse
import itertools


# from solution import build_bridge, get_statistics

STATISTICS = {
    'Artificial_intelligence': [8, 19, 13, 198],
    'Binyamina_train_station_suicide_bombing': [1, 3, 6, 21],
    'Brain': [19, 5, 25, 11],
    'Haifa_bus_16_suicide_bombing': [1, 4, 15, 23],
    'Hidamari_no_Ki': [1, 5, 5, 35],
    'IBM': [13, 3, 21, 33],
    'Iron_Age': [4, 8, 15, 22],
    'London': [53, 16, 31, 125],
    'Mei_Kurokawa': [1, 1, 2, 7],
    'PlayStation_3': [13, 5, 14, 148],
    'Python_(programming_language)': [2, 5, 17, 41],
    'Second_Intifada': [9, 13, 14, 84],
    'Stone_Age': [13, 10, 12, 40],
    'The_New_York_Times': [5, 9, 8, 42],
    'Wild_Arms_(video_game)': [3, 3, 10, 27],
    'Woolwich': [15, 9, 19, 38]}

TESTCASES = (
    ('wiki/', 'Stone_Age', 'Python_(programming_language)',
     ['Stone_Age', 'Brain', 'Artificial_intelligence', 'Python_(programming_language)']),

    ('wiki/', 'The_New_York_Times', 'Stone_Age',
     ['The_New_York_Times', 'London', 'Woolwich', 'Iron_Age', 'Stone_Age']),

    ('wiki/', 'Artificial_intelligence', 'Mei_Kurokawa',
     ['Artificial_intelligence', 'IBM', 'PlayStation_3', 'Wild_Arms_(video_game)',
      'Hidamari_no_Ki', 'Mei_Kurokawa']),

    ('wiki/', 'The_New_York_Times', "Binyamina_train_station_suicide_bombing",
     ['The_New_York_Times', 'Second_Intifada', 'Haifa_bus_16_suicide_bombing',
      'Binyamina_train_station_suicide_bombing']),

    ('wiki/', 'Stone_Age', 'Stone_Age',
     ['Stone_Age', ]),
)


def build_bridge(path, start_page, end_page):
    
    graph = {}
    dist = {start_page: [start_page]}
    q = deque([start_page])
    while len(q):
        at = q.popleft()        
        fpath = os.path.join(path, at)
        if not os.path.isfile(fpath):
            continue
        with open(fpath, encoding="utf-8") as file:
            links = re.findall(r"(?<=/wiki/)[\w()]+", file.read())
            graph[at] = links
        for next in graph[at]:
            if next not in dist:
                dist[next] = [dist[at], next]
                q.append(next)
                
    result1 = dist.get(end_page)
    result = [] 
    def reemovNestings(l): 
        for i in l: 
            if type(i) == list: 
                reemovNestings(i) 
            else: 
                result.append(i) 
    reemovNestings(result1)
    
    return result

def get_statistics(path, start_page, end_page):    
    result = {}
    
    for node in build_bridge(path, start_page, end_page):
        fpath = os.path.join(path, node)
        stats = parse(fpath)
        result[node] = stats
        
    return result


class TestBuildBrige(unittest.TestCase):
    def test_build_bridge(self):
        for path, start_page, end_page, expected in TESTCASES:
            with self.subTest(path=path,
                              start_page=start_page,
                              end_page=end_page,
                              expected=expected):
                result = build_bridge(path, start_page, end_page)
                self.assertEqual(result, expected)


class TestGetStatistics(unittest.TestCase):
    def test_build_bridge(self):
        for path, start_page, end_page, expected in TESTCASES:
            with self.subTest(path=path,
                              start_page=start_page,
                              end_page=end_page,
                              expected=expected):
                result = get_statistics(path, start_page, end_page)
                self.assertEqual(result, {page: STATISTICS[page] for page in expected})


if __name__ == '__main__':
    # unittest.main()
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # Run unit-tests in Jupyter

.

wiki/Stone_Age all nodes count 37
wiki/Brain all nodes count 30
wiki/Artificial_intelligence all nodes count 71
wiki/Python_(programming_language) all nodes count 26
wiki/The_New_York_Times all nodes count 52
wiki/London all nodes count 60
wiki/Woolwich all nodes count 38
wiki/Iron_Age all nodes count 20
wiki/Stone_Age all nodes count 37
wiki/Artificial_intelligence all nodes count 71
wiki/IBM all nodes count 14
wiki/PlayStation_3 all nodes count 38
wiki/Wild_Arms_(video_game) all nodes count 14
wiki/Hidamari_no_Ki all nodes count 12
wiki/Mei_Kurokawa all nodes count 3
wiki/The_New_York_Times all nodes count 52
wiki/Second_Intifada all nodes count 40
wiki/Haifa_bus_16_suicide_bombing all nodes count 6
wiki/Binyamina_train_station_suicide_bombing all nodes count 6
wiki/Stone_Age all nodes count 37



FAIL: test_build_bridge (__main__.TestGetStatistics) (path='wiki/', start_page='The_New_York_Times', end_page='Stone_Age', expected=['The_New_York_Times', 'London', 'Woolwich', 'Iron_Age', 'Stone_Age'])
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-d4529b540632>", line 113, in test_build_bridge
    self.assertEqual(result, {page: STATISTICS[page] for page in expected})
AssertionError: {'The[17 chars] [5, 8, 8, 42], 'London': [53, 16, 31, 125], '[81 chars] 40]} != {'The[17 chars] [5, 9, 8, 42], 'London': [53, 16, 31, 125], '[81 chars] 40]}
  {'Iron_Age': [4, 8, 15, 22],
   'London': [53, 16, 31, 125],
   'Stone_Age': [13, 10, 12, 40],
-  'The_New_York_Times': [5, 8, 8, 42],
?                              ---

+  'The_New_York_Times': [5, 9, 8, 42],
?                            +++

   'Woolwich': [15, 9, 19, 38]}

FAIL: test_build_bridge (__main__.TestGetStatistics) (path='wiki/', start_page='The_New_Y